In [3]:
import pandas as pd
import numpy as np
import elapid
from elapid import MaxentModel
import xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [4]:
#Load environmental data (scaled and no nan)
env_filepath = '/cluster/home/maikents/coral-mapping/processed_data/env_df_no_nan.parquet'
env_df = pd.read_parquet(env_filepath)

In [6]:
#Load presence points data 
coral_filepath = '/cluster/home/maikents/midnor_total_coral_data_processed.parquet'
presence_df = pd.read_parquet(coral_filepath)
presence_coords = presence_df[['x', 'y']].values

print(len(presence_coords))

941


In [7]:
# Generate random pseudoabsence (background) points

#MaxEnt's default setting is to sample 10,000 background
#points regardless of the number of presence points

#pseudoabsence_points = np.random.rand(len(presence_coords), 2) * [max(presence_coords[:,0]), max(presence_coords[:,1])]

# You might want to replace this with a real pseudoabsence generation method.
#pseudoabsence = pd.DataFrame(pseudoabsence_points, columns=['x', 'y'])


# Create a meshgrid for the study area
xx, yy = np.meshgrid(x, y)

# Flatten the grid to create a list of all possible coordinates in the study area
all_coords = pd.DataFrame({
    'x': xx.flatten(),
    'y': yy.flatten()
})


# Exclude presence points from the study area
remaining_coords = pd.merge(all_coords, presence_df, on=['x', 'y'], how='outer', indicator=True)
remaining_coords = remaining_coords[remaining_coords['_merge'] == 'left_only'].drop(columns=['_merge'])

# Randomly sample points from the remaining coordinates
num_samples = 100  # Specify the number of pseudoabsence points
pseudoabsence_points = remaining_coords.sample(n=num_samples, random_state=42)

pseudoabsence = pd.DataFrame(pseudoabsence_points, columns=['x', 'y']).reset_index(drop=True)


KeyboardInterrupt: 

In [ ]:
#Split presence data into train and test sets (80% training, 20% testing)
train_coords, test_coords = train_test_split(presence_coords, test_size=0.2, random_state=42)

#Split the corresponding environmental data into train and test sets
train_env = scaled_environmental_data_df[:len(train_coords)]
test_env = scaled_environmental_data_df[len(train_coords):]

In [ ]:
#Model training
#Initialize the MaxEnt model
maxent_model = elapid.MaxentModel()
#maxent_model = MaxEnt()

#Train the model using the training data (environmental variables and species presence coordinates)
maxent_model.fit(train_env, train_coords)


"""
model = elapid.MaxentModel(
    feature_types = ['linear', 'hinge', 'product'], # the feature transformations
    tau = 0.5, # prevalence scaler
    clamp = True, # set covariate min/max based on range of training data
    scorer = 'roc_auc', # metric to optimize (from sklearn.metrics.SCORERS)
    beta_multiplier = 1.0, # regularization scaler (high values drop more features)
    beta_lqp = 1.0, # linear, quadratic, product regularization scaler
    beta_hinge = 1.0, # hinge regularization scaler
    beta_threshold = 1.0, # threshold regularization scaler
    beta_categorical = 1.0, # categorical regularization scaler
    n_hinge_features = 50, # number of hinge features to compute
    n_threshold_features = 50, # number of threshold features to compute
    convergence_tolerance = 1e-07, # model fit convergence threshold
    use_lambdas = 'last', # set to 'best' (least overfit), 'last' (highest score)
    n_cpus = 4, # number of cpu cores to use
)
"""

In [ ]:
#Make predictions on the test set

test_predictions = maxent_model.predict(test_env)
